In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-11-27 23:18
@Project  :   Hands-on Crawler with Python-process
进程
'''

# 导入所需的库
import os
import random
import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from multiprocessing import Process, Pool, Lock, RLock, Queue

# 1.multiprocessing库
Python中的多进程是通过multiprocessing库来实现的，和多线程的threading.Thread差不多，它可以利用multiprocessing.Process类来创建一个进程对象，其使用方法与threading库很像。

Python进程在不同的系统启动方式不同：
- spawn：父进程会启动一个全新的Python解释器进程，子进程会继承那些运行进程对象的run()方法所必需的资源。使用此方法启动进程相比使用fork或forkserver要慢上许多，在Unix、Windows、MacOS上使用，同时是Windows的默认设置。
- fork：父进程和子进程共享内存，但互不影响，子进程会继承父进程的运行状态。使用此方法启动进程相比使用spawn要快很多，在Unix上使用。
- forkserver：程序选择forkserver启动方法时，每当需要一个新进程时，父进程就会连接到服务器并请求它分叉一个新进程，分叉服务器进程是单线程的，因此使用os.fork()是安全的，没有不必要的资源被继承。可在Unix平台上使用，支持通过Unix管道传递文件描述符。

Windows默认使用spawn方式创建新进程，而Unix（Linux）使用fork方式创建新进程。同时，因为Windows没有fork机制，只能使用spawn方式，而该方法会先创建出一个子进程，为了获得相同的功能，子进程会再次运行代码，此时代码中需要有__main__来区分父进程和子进程。下面的示例都是基于Linux来实现的，如果想要在Windows实现，应该使用Python脚本文件而不是Jupyter Notebook等交互式环境，同时应该将主线程用`if __name__ == '__main__':`进行包裹。

In [2]:
# 方式1：直接实例化Process类
def sleep_for_seconds(name):
    work_time = random.random() * 5
    time.sleep(work_time)
    print(f'{name} worker finished in {work_time:.4f} seconds')


tasks = []
for i in range(5):
    p = Process(target=sleep_for_seconds, args=(f'process-{i}',))
    p.start()
    tasks.append(p)

for task in tasks:
    task.join()

print('Main Done')

process-3 worker finished in 0.5238 seconds
process-4 worker finished in 0.8303 seconds
process-2 worker finished in 1.2532 seconds
process-1 worker finished in 3.4665 seconds
process-0 worker finished in 4.2249 seconds
Main Done


In [3]:
for i in range(5):
    p = Process(target=sleep_for_seconds, args=(f'process-{i}',))
    p.start()
    p.join()  # 每一个进程在start之后调用join

print('Main Done')  # 速度变慢，由并行变为串行，因为join()方法将阻塞直到调用该方法的进程终止。一个进程可以被join多次；进程无法join自身，因为这会导致死锁

process-0 worker finished in 0.8161 seconds
process-1 worker finished in 0.5090 seconds
process-2 worker finished in 2.2470 seconds
process-3 worker finished in 0.4468 seconds
process-4 worker finished in 1.6233 seconds
Main Done


In [4]:
# 方式2：继承Process类，重写run()方法
class CustomProcess(Process):
    def __init__(self, func, name):
        super().__init__()
        self.func = func
        self.name = name

    def run(self):
        self.func(self.name)


tasks = []
for i in range(5):
    p = CustomProcess(sleep_for_seconds, f'process-{i}')
    p.start()
    tasks.append(p)

for task in tasks:
    task.join()

print('Main Done')  # 效果与方式1相同

process-0 worker finished in 0.5188 seconds
process-1 worker finished in 1.9588 seconds
process-4 worker finished in 2.4376 seconds
process-3 worker finished in 3.0578 seconds
process-2 worker finished in 3.3478 seconds
Main Done


# 2.进程池

In [5]:
def sleep_for_seconds(name):
    print(f'Run task {name} {os.getpid()}')
    work_time = random.random() * 5
    time.sleep(work_time)
    print(f'{name} worker finished in {work_time:.4f} seconds')


print(f'Run main {os.getpid()}')
pool = Pool(3)
for i in range(5):
    pool.apply_async(sleep_for_seconds, args=(f'process-{i}',))
pool.close()  # join钱必须调用close()，调用close()之后不能继续添加新的进程
pool.join()  # 调用join()方法会等待所有子进程执行完毕
print('Main Done')

Run main 2114
Run task process-0 2207Run task process-1 2208Run task process-2 2209


process-1 worker finished in 0.2262 seconds
Run task process-3 2208
process-0 worker finished in 2.2371 seconds
Run task process-4 2207
process-2 worker finished in 3.3186 seconds
process-3 worker finished in 3.1034 seconds
process-4 worker finished in 3.0111 seconds
Main Done


# 3.进程锁

In [6]:
def sleep_for_seconds(i, lock):
    try:
        lock.acquire()  # 获取锁：控制一段代码在同一时间只能被一个进程执行，如果在locked得锁上再次获取，会阻塞进程
        print(f'{i}: {os.getpid()} is running')
        time.sleep(random.random() * 5)
        print(f'{i}: {os.getpid()} is done')
    finally:
        lock.release()  # 释放锁


lock = Lock()
for i in range(10):
    Process(target=sleep_for_seconds, args=(i, lock)).start()  # 使用锁保证进程启动顺序和结束顺序一致，一个进程运行结束才开始运行下一个

0: 2255 is running
0: 2255 is done
1: 2256 is running
1: 2256 is done
2: 2261 is running
2: 2261 is done
3: 2262 is running
3: 2262 is done
4: 2263 is running
4: 2263 is done
5: 2264 is running
5: 2264 is done
6: 2265 is running
6: 2265 is done
7: 2266 is running
7: 2266 is done
8: 2267 is running
8: 2267 is done
9: 2268 is running
9: 2268 is done


In [7]:
# 使用with语句替代
def sleep_for_seconds(i, lock):
    with lock:
        print(f'{i}: {os.getpid()} is running')
        time.sleep(random.random() * 5)
        print(f'{i}: {os.getpid()} is done')


lock = Lock()
for i in range(10):
    Process(target=sleep_for_seconds, args=(i, lock)).start()  # 效果相同

0: 2269 is running
0: 2269 is done
1: 2270 is running
1: 2270 is done
2: 2275 is running
2: 2275 is done
3: 2276 is running
3: 2276 is done
4: 2277 is running
4: 2277 is done
5: 2278 is running
5: 2278 is done
6: 2279 is running
6: 2279 is done
7: 2280 is running
7: 2280 is done
8: 2281 is running
8: 2281 is done
9: 2282 is running
9: 2282 is done


In [8]:
# 可重入锁
def sleep_for_seconds(i, lock):
    try:
        lock.acquire()  # 获取锁
        lock.acquire()  # 再次获取锁
        print(f'{i}: {os.getpid()} is running')
        time.sleep(random.random() * 5)
        print(f'{i}: {os.getpid()} is done')
    finally:
        lock.release()  # 释放锁
        lock.release()  # 再次释放锁


lock = RLock()
for i in range(10):
    Process(target=sleep_for_seconds, args=(i, lock)).start()

0: 2283 is running
0: 2283 is done
1: 2284 is running
1: 2284 is done
2: 2287 is running
2: 2287 is done
3: 2288 is running
3: 2288 is done
4: 2291 is running
4: 2291 is done
5: 2292 is running
5: 2292 is done
6: 2293 is running
6: 2293 is done
7: 2294 is running
7: 2294 is done
8: 2295 is running
8: 2295 is done
9: 2296 is running
9: 2296 is done


# 4.进程通信

In [9]:
# Queue实现进程间通信
chat_names = ['ChatGPT', 'ERNIE Bot', 'QianWen', 'iFLYTEK Spark', 'ChatGLM']


# 进程写数据
def write(q):
    print(f'Process {os.getpid()} is writing...')
    for value in chat_names:
        print(f'Put {value} into queue...')
        q.put(value)
        time.sleep(random.random() * 5)


# 进程读数据
def read(q):
    print(f'Process {os.getpid()} is reading...')
    while True:
        value = q.get()
        print(f'Get {value} from queue...')


# 父进程创建队列，并传给2个子进程
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
pw.start()  # 启动写进程
pr.start()  # 启动读进程
pw.join()  # 等待写进程结束
pr.kill()  # 强行终止进程

Process 2297 is writing...
Put ChatGPT into queue...
Process 2298 is reading...
Get ChatGPT from queue...
Put ERNIE Bot into queue...
Get ERNIE Bot from queue...
Put QianWen into queue...
Get QianWen from queue...
Put iFLYTEK Spark into queue...
Get iFLYTEK Spark from queue...
Put ChatGLM into queue...
Get ChatGLM from queue...


In [10]:
# 通信内容为自定义类实例
class CustomClass:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return f'CustomClass({self.name})'

    def __repr__(self):
        return f'CustomClass({self.name})'


def write(q):
    print(f'Process {os.getpid()} is writing...')
    for value in [CustomClass(name) for name in chat_names]:
        print(f'Put {value} into queue...')
        q.put(value)
        time.sleep(random.random() * 5)


def read(q):
    print(f'Process {os.getpid()} is reading...')
    while True:
        value = q.get()
        print(f'Get {value} from queue...')


q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
pw.start()
pr.start()
pw.join()
pr.kill()  # 正常运行

Process 2479 is writing...
Put CustomClass(ChatGPT) into queue...Process 2480 is reading...

Get CustomClass(ChatGPT) from queue...
Put CustomClass(ERNIE Bot) into queue...
Get CustomClass(ERNIE Bot) from queue...
Put CustomClass(QianWen) into queue...
Get CustomClass(QianWen) from queue...
Put CustomClass(iFLYTEK Spark) into queue...
Get CustomClass(iFLYTEK Spark) from queue...
Put CustomClass(ChatGLM) into queue...
Get CustomClass(ChatGLM) from queue...


In [11]:
# 通信内容为自定义类实例，同时在类中使用锁：不支持，因为锁对象只能通过传参在进程之间共享，不能通过进程通信的形式传输
class CustomClass:
    def __init__(self, name):
        self.name = name
        self.lock = Lock()

    def __str__(self):
        return f'CustomClass({self.name})'

    def __repr__(self):
        return f'CustomClass({self.name})'


def write(q):
    print(f'Process {os.getpid()} is writing...')
    for value in [CustomClass(name) for name in chat_names]:
        print(f'Put {value} into queue...')
        q.put(value)
        time.sleep(random.random() * 5)


def read(q):
    print(f'Process {os.getpid()} is reading...')
    while True:
        value = q.get()
        print(f'Get {value} from queue...')


q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
pw.start()
pr.start()
pw.join()
pr.kill()  # 报错：RuntimeError: Lock objects should only be shared between processes through inheritance

Process 2607 is writing...
Put CustomClass(ChatGPT) into queue...Process 2608 is reading...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/synchronize.py", line 101, in __getstate__
    context.assert_spawning(self)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/context.py", line 359, in assert_spawning
    raise RuntimeError(
RuntimeError: Lock objects should only be shared between processes through inheritance


Put CustomClass(ERNIE Bot) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/synchronize.py", line 101, in __getstate__
    context.assert_spawning(self)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/context.py", line 359, in assert_spawning
    raise RuntimeError(
RuntimeError: Lock objects should only be shared between processes through inheritance


Put CustomClass(QianWen) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/synchronize.py", line 101, in __getstate__
    context.assert_spawning(self)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/context.py", line 359, in assert_spawning
    raise RuntimeError(
RuntimeError: Lock objects should only be shared between processes through inheritance


Put CustomClass(iFLYTEK Spark) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/synchronize.py", line 101, in __getstate__
    context.assert_spawning(self)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/context.py", line 359, in assert_spawning
    raise RuntimeError(
RuntimeError: Lock objects should only be shared between processes through inheritance


Put CustomClass(ChatGLM) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/synchronize.py", line 101, in __getstate__
    context.assert_spawning(self)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/context.py", line 359, in assert_spawning
    raise RuntimeError(
RuntimeError: Lock objects should only be shared between processes through inheritance


In [12]:
# 通信内容为自定义类实例，同时在类中使用lambda匿名函数：不支持，因为Queue的实现原理是，当一个对象要被放入队列中时，这个对象首先会被一个后台线程用pickle 序列化，并将序列化后的数据通过一个底层管道的管道传递到队列中，而lambda不支持被序列化
class CustomClass:
    def __init__(self, name):
        self.name = name
        self.func = lambda x: len(x)

    def __str__(self):
        return f'CustomClass({self.name})'

    def __repr__(self):
        return f'CustomClass({self.name})'


def write(q):
    print(f'Process {os.getpid()} is writing...')
    for value in [CustomClass(name) for name in chat_names]:
        print(f'Put {value} into queue...')
        q.put(value)
        time.sleep(random.random() * 5)


def read(q):
    print(f'Process {os.getpid()} is reading...')
    while True:
        value = q.get()
        print(f'Get {value} from queue...')


q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
pw.start()
pr.start()
pw.join()
pr.kill()  # 报错：AttributeError: Can't pickle local object 'CustomClass.__init__.<locals>.<lambda>'

Process 2726 is writing...
Put CustomClass(ChatGPT) into queue...
Process 2727 is reading...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'CustomClass.__init__.<locals>.<lambda>'


Put CustomClass(ERNIE Bot) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'CustomClass.__init__.<locals>.<lambda>'


Put CustomClass(QianWen) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'CustomClass.__init__.<locals>.<lambda>'


Put CustomClass(iFLYTEK Spark) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'CustomClass.__init__.<locals>.<lambda>'


Put CustomClass(ChatGLM) into queue...


Traceback (most recent call last):
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/local/miniconda3/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'CustomClass.__init__.<locals>.<lambda>'


# 5.进程池ProcessPoolExecutor

In [13]:
# 使用进程池
def sum_numbers(n):
    res = 0
    for i in range(n):
        res += i
    return res


nums = [1, 10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000]
with ProcessPoolExecutor(max_workers=3) as executor:  # 进程池
    tasks = [executor.submit(sum_numbers, num) for num in nums]
    for task in as_completed(tasks):
        print(task.result())

0
45
4950
499500
49995000
4999950000
499999500000
49999995000000
4999999950000000
499999999500000000


In [14]:
# 进程池与线程池的比较
# 顺序执行
start = time.perf_counter()
for num in nums:
    res = sum_numbers(num)
    print(res)
print(f'Sequential execution in {time.perf_counter() - start:.4f} seconds')

# 线程池执行
start = time.perf_counter()
with ThreadPoolExecutor(max_workers=3) as executor:
    tasks = [executor.submit(sum_numbers, num) for num in nums]
    for task in as_completed(tasks):
        print(task.result())
print(f'Thread pool execution in {time.perf_counter() - start:.4f} seconds')

# 进程池执行
start = time.perf_counter()
with ProcessPoolExecutor(max_workers=3) as executor:
    tasks = [executor.submit(sum_numbers, num) for num in nums]
    for task in as_completed(tasks):
        print(task.result())
print(
    f'Process pool execution in {time.perf_counter() - start:.4f} seconds')  # 相比于线程池，ProcessPoolExecutor使用了多核处理的模块，使得可以不受GIL的限制，大大缩短执行时间

0
45
4950
499500
49995000
4999950000
499999500000
49999995000000
4999999950000000
499999999500000000
Sequential execution in 87.8482 seconds
499500
49995000
45
4950
0
4999950000
499999500000
49999995000000
4999999950000000
499999999500000000
Thread pool execution in 105.0270 seconds
0
4950
499500
45
49995000
4999950000
499999500000
49999995000000
4999999950000000
499999999500000000
Process pool execution in 81.6088 seconds


线程和进程是操作系统中的两个基本概念，它们在Python中也有所体现。

关系：
- 一个进程可以包含多个线程，这些线程共享进程的资源，如内存、文件等。
- 线程相对于进程来说，创建、销毁和切换的开销更小，因此多线程可以提高程序的执行效率。
- 线程之间可以通过同步机制（如锁、信号量等）进行通信和协作。

区别：
- 资源分配：进程是资源分配的基本单位，每个进程拥有独立的内存空间和系统资源；而线程是CPU调度的基本单位，多个线程共享同一个进程的资源。
- 独立性：进程之间是独立的，一个进程崩溃不会影响到其他进程；而线程之间是共享资源的，一个线程崩溃可能会导致整个进程崩溃。
- 通信方式：进程间通信需要借助IPC（进程间通信）机制，如管道、消息队列等；而线程间通信相对简单，可以直接访问共享变量、全局变量等。
- 数据传递：由于进程之间的内存空间是独立的，因此进程间的数据传递需要通过IPC机制；而线程共享同一进程的内存空间，数据传递相对简单。
- 内存管理：每个进程拥有独立的内存空间，需要单独进行内存分配和回收；而线程共享同一进程的内存空间，内存管理相对简单。
- 上下文切换：进程间的上下文切换开销较大，因为涉及到内存、CPU等资源的分配和回收；而线程间的上下文切换开销较小，因为它们共享同一进程的资源。
- 编程模型：多进程编程需要考虑进程间的同步和互斥问题，通常使用multiprocessing模块；而多线程编程需要考虑线程间的同步和互斥问题，通常使用threading模块。